In [0]:
# Gold: FX-adjusted import costs, trends & volatility
# Inputs:  fx_impact.silver_monthly_fact
# Outputs:
#   fx_impact.gold_monthly_metrics        (month × HS-2 with MoM/YoY, rolling vol, shares)
#   fx_impact.gold_period_summary         (Pre/During/Post COVID deltas vs Pre)
#   fx_impact.gold_monthly_totals         (monthly totals across HS-2)


In [0]:
# Cell 1 — Ensure schema
spark.sql("CREATE DATABASE IF NOT EXISTS fx_impact")


In [0]:
# Cell 2 — Month dimension (fills gaps cleanly)
spark.sql("""
CREATE OR REPLACE TEMP VIEW bounds AS
SELECT MIN(month) AS min_m, MAX(month) AS max_m
FROM fx_impact.silver_monthly_fact
""")

spark.sql("""
CREATE OR REPLACE TABLE fx_impact.dim_months AS
WITH b AS (SELECT * FROM bounds)
SELECT explode(sequence(b.min_m, b.max_m, interval 1 month)) AS month
FROM b
""")


In [0]:
# Cell 3 — Base series (complete month × HS-2 grid, left-join Silver)
spark.sql("""
CREATE OR REPLACE TEMP VIEW base_series AS
WITH c AS (
  SELECT DISTINCT cmdCode, COALESCE(cmdDesc, cmdCode) AS cmdDesc
  FROM fx_impact.silver_monthly_fact
)
SELECT
  d.month,
  c.cmdCode,
  c.cmdDesc
FROM fx_impact.dim_months d
CROSS JOIN c
""")

spark.sql("""
CREATE OR REPLACE TEMP VIEW joined AS
SELECT
  b.month,
  b.cmdCode,
  b.cmdDesc,
  s.import_usd,
  s.import_eur,
  s.USD_per_EUR, s.JPY_per_EUR, s.CNY_per_EUR,
  s.usd_obs_days, s.usd_days_total, s.usd_null_share,
  s.covid_period
FROM base_series b
LEFT JOIN fx_impact.silver_monthly_fact s
  ON b.month = s.month AND b.cmdCode = s.cmdCode
""")


We now have every month × commodity, even if a month was missing in Comtrade (those EUR/USD values will be NULL → treated as 0 for totals, but preserved for MoM logic).

In [0]:
# Cell 4 — Per-commodity metrics (MoM/YoY, rolling vol, shares)
spark.sql("""
CREATE OR REPLACE TABLE fx_impact.gold_monthly_metrics AS
WITH z AS (
  SELECT
    month, cmdCode, cmdDesc,
    COALESCE(import_eur, 0.0) AS import_eur,
    COALESCE(import_usd, 0.0) AS import_usd,
    USD_per_EUR, JPY_per_EUR, CNY_per_EUR,
    usd_obs_days, usd_days_total, usd_null_share,
    covid_period,
    -- Totals across commodities for shares
    SUM(COALESCE(import_eur,0.0)) OVER (PARTITION BY month) AS total_eur_month
  FROM joined
),
w AS (
  SELECT
    z.*,
    LAG(import_eur, 1)  OVER (PARTITION BY cmdCode ORDER BY month) AS eur_lag_1,
    LAG(import_eur, 12) OVER (PARTITION BY cmdCode ORDER BY month) AS eur_lag_12,
    LAG(USD_per_EUR, 1) OVER (PARTITION BY cmdCode ORDER BY month) AS usd_per_eur_lag_1,

    -- Rolling windows (ROWS, not RANGE → strict lookback)
    STDDEV_POP(import_eur) OVER (
      PARTITION BY cmdCode ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS eur_vol_3m,

    STDDEV_POP(import_eur) OVER (
      PARTITION BY cmdCode ORDER BY month
      ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) AS eur_vol_6m,

    STDDEV_POP(USD_per_EUR) OVER (
      PARTITION BY cmdCode ORDER BY month
      ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS fx_vol_3m,

    STDDEV_POP(USD_per_EUR) OVER (
      PARTITION BY cmdCode ORDER BY month
      ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
    ) AS fx_vol_6m
  FROM z
)
SELECT
  month, cmdCode, cmdDesc, covid_period,
  import_usd, import_eur,
  USD_per_EUR, JPY_per_EUR, CNY_per_EUR,
  usd_obs_days, usd_days_total, usd_null_share,
  total_eur_month,
  CASE WHEN total_eur_month > 0 THEN import_eur / total_eur_month END AS share_of_total_eur,

  -- MoM & YoY for EUR imports
  CASE WHEN eur_lag_1  > 0 THEN (import_eur - eur_lag_1)  / eur_lag_1  END AS eur_mom_pct,
  CASE WHEN eur_lag_12 > 0 THEN (import_eur - eur_lag_12) / eur_lag_12 END AS eur_yoy_pct,

  -- FX MoM (% change in USD_per_EUR)
  CASE WHEN usd_per_eur_lag_1 > 0 THEN (USD_per_EUR - usd_per_eur_lag_1) / usd_per_eur_lag_1 END AS fx_mom_pct,

  -- Rolling volatility (level-based; optional: swap to log-return vol later)
  eur_vol_3m, eur_vol_6m, fx_vol_3m, fx_vol_6m
FROM w
""")


In [0]:
# Cell 5 — Monthly totals table (nice for quick cards/lines)
spark.sql("""
CREATE OR REPLACE TABLE fx_impact.gold_monthly_totals AS
SELECT
  month,
  SUM(import_usd) AS total_import_usd,
  SUM(import_eur) AS total_import_eur,
  AVG(USD_per_EUR) AS avg_usd_per_eur
FROM fx_impact.gold_monthly_metrics
GROUP BY month
""")


In [0]:
# Cell 6 — Period summary (Pre/During/Post vs Pre deltas)
spark.sql("""
CREATE OR REPLACE TABLE fx_impact.gold_period_summary AS
WITH base AS (
  SELECT covid_period,
         AVG(import_eur) AS avg_monthly_eur
  FROM fx_impact.gold_monthly_metrics
  GROUP BY covid_period
),
p AS (
  SELECT
    MAX(CASE WHEN covid_period='Pre-COVID'    THEN avg_monthly_eur END) AS pre,
    MAX(CASE WHEN covid_period='During COVID' THEN avg_monthly_eur END) AS during,
    MAX(CASE WHEN covid_period='Post-COVID'   THEN avg_monthly_eur END) AS post
  FROM base
)
SELECT
  pre  AS avg_eur_pre,
  during AS avg_eur_during,
  post AS avg_eur_post,
  CASE WHEN pre > 0 THEN (during - pre) / pre END AS delta_during_vs_pre_pct,
  CASE WHEN pre > 0 THEN (post   - pre) / pre END AS delta_post_vs_pre_pct
FROM p
""")


In [0]:
# Cell 7 — Sanity preview
spark.sql("SELECT * FROM fx_impact.gold_monthly_metrics ORDER BY month, cmdCode LIMIT 20").show()
spark.sql("SELECT * FROM fx_impact.gold_monthly_totals ORDER BY month LIMIT 12").show()
spark.sql("SELECT * FROM fx_impact.gold_period_summary").show()


Notes / rationale

Filling the grid ensures consistent windows (3M/6M) even if a commodity had a missing month; we treat missing imports as 0 for totals and volatility. If you prefer to keep gaps, remove the COALESCE and adjust logic (MoM/YoY will be NULL where history is missing).

Volatility here is std-dev of the levels. For finance-style volatility of returns, replace with STDDEV_POP(ln(import_eur / LAG(import_eur))) over the same rolling window (guard zeros).

Shares use month total from the same table (window aggregate), so Power BI can render a contribution bar for each month.

DAX suggestions (when you wire to Gold)

Cards: Total EUR, MoM %, Δ Post vs Pre (from gold_period_summary), Avg USD_per_EUR.

Line: total_import_eur over Date[Date] (from gold_monthly_totals).

Stacked area: import_eur by cmdDesc over time (from gold_monthly_metrics).

Bar + line: Contribution (share_of_total_eur) with avg_usd_per_eur.